# Using [*Augmented*](https://arxiv.org/abs/1904.01681) [*Neural ODEs*](https://arxiv.org/abs/1806.07366) for ***real-world*** [irregularly-sampled time-series](https://arxiv.org/abs/1907.03907) prediction and simulation.


A joint effort of

* [Arianna Tasciotti](https://github.com/ariannatasciotti)  
* [Milton Nicolás Plasencia Palacios](https://github.com/nickplas)  
* [Emanuele Ballarin](https://ballarin.cc/)

and probably part of [this GitHub repository](https://github.com/emaballarin/cathode).  

An upfront *thank you* to [Davide Scassola](https://github.com/DavideScassola), for having made this task a task worth trying to attack. *Per adversa, ultra adversa.*

**NOTICE:**  
This Colab/Jupyter notebook just contains the *neural network part* of the analysis. The starting point for this notebook is an `HDF5` file containing the preprocessed dataframe, ready to be used. There should be a notebook ready just for that!

## Package installation
Work-around *Colab* incompatibilities.  
*(and remember to restart the runtime at the end!)*

## Reproducibility & compatibility information

A little help to foster code reproducibility, courtesy of [Sebastian Raschka](http://sebastianraschka.com/)'s [`Watermark`](https://github.com/rasbt/watermark).

In [ ]:
# Be nice, be reproducible!
%reload_ext watermark
%watermark -a 'A. Tasciotti, M.N. Plasencia Palacios, and E. Ballarin' -v -w -p numpy,vaex,torch,torchdiffeq

## Modules

Keeping all imported modules in one single place.

In [ ]:
## IMPORTS ##

# Numpy first! (required to force an Intel OpenMP threading layer)
import numpy as np

# Basics
import os

# Scientific computing
import numpy as np
import scipy as sp

# Data handling
import vaex as vx

# Plotting
import matplotlib
import matplotlib.pyplot as plt

# Neural Networks / Neural ODEs
import torch
import torchdiffeq as thdeq
import torch.nn as nn
from torchdiffeq import odeint
import torch.nn.functional as F
import torch.distributions as distributions
import torch.utils.data.dataloader as DataLoader
from torch import optim
import torch.cuda.amp as amp
from torch._six import inf
from torchsummary import summary

# Self-rolled utility functions
from src.util.datamanip import data_by_tick, data_by_tick_col
from src.util.plotting import data_timeplot
from src.util.datasets import StockDataset
from pyromaniac.optim.torch.adamwcd import AdamWCD as AdamWCD 

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device: {}'.format(device))

## Data import

Starting from where we left.

In [ ]:
# HYPERPARAMETERS
batch_size = 64
input_datasize = 1
hidden_size_gruode = 16*(input_datasize+1)
hidden_size_ffw_f = 180
hidden_size_ffw_g = 30
hidden_size_ffw_onn = 30
output_datasize = input_datasize
ttsr = 0.9
window_size = 1000
max_train_len = 20

In [ ]:
class f(torch.nn.Module):
  def __init__(self, size, hidden_size):
    super(f, self).__init__()
    self.fc1 = nn.Linear(size+1, hidden_size)
    self.fc2 = nn.Linear(hidden_size, hidden_size)
    self.fc3 = nn.Linear(hidden_size, size)

  def forward(self, t, data):
    x = torch.cat((torch.tensor([t],requires_grad=True).to(device),data.to(device)), dim=0).to(device)
    x.retain_grad()
    x = self.fc1(x)
    x = torch.tanh(x)
    x = self.fc2(x)
    x = torch.tanh(x)
    x = self.fc3(x)
    x = torch.tanh(x)
    return x

In [ ]:
class CustomOdeint(torch.nn.Module):
  def __init__(self, size, hidden_size):
    super(CustomOdeint, self).__init__()
    self.size = size
    self.hidden_size = hidden_size
    self.f_forward = f(self.size, self.hidden_size).to(device)

  def forward(self, y0, t, rtol=1e-7, atol=1e-9, method=None, options=None):
    b_size = len(y0)
    assert b_size == len(t)
    output = torch.Tensor(b_size,t.size()[1],self.size).to(device)
    for i in range(0,b_size):
      output[i] = odeint(self.f_forward, y0[i], t[i], rtol, atol, method, options)
    return output

In [ ]:
customodeint = CustomOdeint(hidden_size_gruode,hidden_size_ffw_f)

In [ ]:
class GRUODE(torch.nn.Module):
  def __init__(self, input_size, hidden_size, bias=True):  # hidden_size == output_size of Sampler class
    super(GRUODE, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.bias = bias
    self.GRU = torch.nn.GRUCell(self.input_size, self.hidden_size, self.bias).to(device)
    self.LN = torch.nn.LayerNorm(hidden_size)


  def forward(self, x, hidden, times):
    # GRU training algorithm
    h_n = customodeint(hidden.to(device), times.to(device), rtol = 1e-3, atol = 1e-4, method='dopri5')[:,1] # check
    hy = self.LN(h_n)
    hy = self.GRU(x, h_n)
    return hy

In [ ]:
class g(torch.nn.Module):
  def __init__(self, input_size, hidden_size, output_size=2):  # input_size == hidden_size of GRUODE
    super(g, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.fc2 = nn.Linear(hidden_size, hidden_size)
    self.fc3 = nn.Linear(hidden_size, output_size)
    self.BN =  nn.BatchNorm1d(hidden_size)
    self.output_size = output_size

  def forward(self, x):
    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    x = self.BN(x)
    x = self.fc3(x)
    #if x[1] <= 0:
      #x = torch.tensor([x[0],-x[1]], requires_grad=True).to(device)
    x = torch.cat((x[:,0], torch.abs(x[:,1])), dim=0).to(device).view(-1, self.output_size)
    return x 

In [ ]:
class Sampler(torch.nn.Module):
  def __init__(self, dist, nr_parameters, output_size):             # dim nr_parameters != dim output_size
    super(Sampler, self).__init__()
    self.dist = dist
    self.nr_parameters = nr_parameters
    self.output_size = output_size

  def forward(self, parameters):
    parameters = parameters.t()
    parlist = parameters.chunk(self.nr_parameters)
    distribution = self.dist(*parlist)                                # unpacking elements of parlist
    out =  distribution.rsample(torch.tensor([self.output_size]))[:,0]     # non vuole requires_grad()
    out = out.t()                                                     
    return out

In [ ]:
class OutputNN(torch.nn.Module):
  def __init__(self, input_size, hidden_size, output_size): # dim input_size == hidden_size of GRUODE, dim output_size == dim predictions
    super(OutputNN, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.fc2 = nn.Linear(hidden_size, hidden_size)
    self.fc3 = nn.Linear(hidden_size, output_size)

  def forward(self, x):
    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    x = self.fc3(x)
    return x  

In [ ]:
class RNNVAEODE(torch.nn.Module):
  def __init__(self, input_size, hidden_size_gruode, hidden_size_ffw_g, hidden_size_ffw_onn, prior, h0): 
    super(RNNVAEODE, self).__init__()

    self.h0 = h0
    self.h0_buffer = h0
    self.GRUODE = GRUODE(input_size, hidden_size_gruode, bias=True)
    self.g = g(hidden_size_gruode, hidden_size_ffw_g ,output_size=2) 
    self.Sampler = Sampler(prior, 2, hidden_size_gruode)
    self.OutputNN = OutputNN(hidden_size_gruode, hidden_size_ffw_onn, input_size)
 
  def set_h0(self, new_h0):
    self.h0 = new_h0.detach().clone()
    self.h0_buffer = new_h0.detach().clone()

  def forward(self, past, t_future):

    self.h0 = self.h0_buffer.detach().clone()

    if past is not None:  #training
      past_time = past[:,:,:1]
      past_data = past[:,:,1:]
      

      h_new = self.GRUODE(past_data[:,0],self.h0, torch.cat((past_time[:,0]-torch.ones(past_time[:,0].size()).to(device), past_time[:,0]), dim=1).to(device))
      h_prev = h_new

      for i in range(1,len(past)):
        h_new = self.GRUODE(past_data[:,i], h_prev, torch.cat((past_time[:,i-1], past_time[:,i]), dim=1).to(device))
        h_prev = h_new

      self.h0_buffer = h_prev.detach().clone()

    else:
      h_prev = self.h0                        # not rolled

    if t_future is None:                      
      return None                             # not rolled


    param = self.g(h_prev)
    z0 = self.Sampler(param)
    out = customodeint(z0, t_future, rtol = 1e-3, atol = 1e-4, method='dopri5')[:,:]
    output = self.OutputNN(out)
    return output

In [ ]:
# INITIAL HIDDEN STATE
h0 = torch.zeros(batch_size, hidden_size_gruode)
h0.to(device)

In [ ]:
h0_test = torch.zeros(1, hidden_size_gruode)
h0_test.to(device)

In [ ]:
mydataset = StockDataset("./data/WIKI_PRICES_QUANDL.hdf5", 'AAPL', 'close', ttsr, window_size, batch_size=batch_size)

In [ ]:
mydataset_test = StockDataset("./data/WIKI_PRICES_QUANDL.hdf5", 'AAPL', 'close', ttsr, window_size, train=False) 

In [ ]:
model = RNNVAEODE(input_datasize, hidden_size_gruode, hidden_size_ffw_g, hidden_size_ffw_onn, distributions.Normal, h0)
model.to(device)

In [ ]:
dataloader = torch.utils.data.DataLoader(mydataset, batch_size=batch_size)

In [ ]:
dataloader_test = torch.utils.data.DataLoader(mydataset_test, batch_size=1)

In [ ]:
 def param_gn(parameters, norm_type=2): 
  if isinstance(parameters, torch.Tensor):
      parameters = [parameters]
  parameters = list(filter(lambda p: p.grad is not None, parameters))
  norm_type = float(norm_type)
  if len(parameters) == 0:
      return torch.tensor(0.)
  device = parameters[0].grad.device
  if norm_type == inf:
      total_norm = max(p.grad.detach().abs().max().to(device) for p in parameters)
  else:
      total_norm = torch.norm(torch.stack([torch.norm(p.grad.detach(), norm_type).to(device) for p in parameters]), norm_type)
  return total_norm

In [ ]:
def get_params_num(net):
    return sum(map(torch.numel, net.parameters()))

In [ ]:
print(get_params_num(model))

In [ ]:
# TRAINING LOOP 
lr = 0.00205
#momentum = 0.9
eps = 1e-8
lrd = 1.0
cn = 63*batch_size
out_cn = 3*cn
wd = 0.005
epochs = 20

n_batches = len(dataloader)

criterion = nn.SmoothL1Loss(reduction = 'sum')
#criterion = nn.MSELoss()
#optimizer = AdamWCD(model.parameters(), lr=lr, eps=eps, clip_norm=cn, lrd=lrd)
#optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
optimizer = optim.AdamW(model.parameters(), lr=lr, eps=eps, weight_decay=wd)

model.train() 
for e in range(epochs):
  for i,dictionary in enumerate(dataloader):
    clip_iter = 3
    optimizer.zero_grad()
    data_in = dictionary["past"].float().to(device)
    data_out_time = dictionary["future"][:,:,:1].float().to(device)
    data_out_data = dictionary["future"][:,:,1:].float().to(device)
    # tutto quello che viene valutato nel forward viene castato a float.32, 
    # mentre tutte le cose che vengono calcolate solo una volta (gradienti...) vengono castati a float.16
    #with torch.cuda.amp.autocast(enabled=True):                   
    outputs = model(data_in,data_out_time)
    loss = criterion(outputs, data_out_data)
    #optimizer.zero_grad()
    loss.backward()
    paramgn = param_gn(model.parameters())
    print("gradient_norm: ", paramgn.item())
    if (paramgn > out_cn):
      clip_iter = 2 
    if (i % clip_iter == 0):
      torch.nn.utils.clip_grad_norm_(model.parameters(), cn)
    optimizer.step()
    if i % 1 == 0:
        print("[EPOCH]: {}, [BATCH]: {}/{}, [LOSS]: {}".format(e, i, n_batches, loss.item()))
        print(" ")

In [ ]:
# Save the model
torch.save(model.state_dict(), "RNNVAEODE_bkp.pt")

In [ ]:
model_test = model
model_test.eval() 
model_test.set_h0(h0_test)
criterion = nn.MSELoss()
for i,dictionary in enumerate(dataloader_test):
  if dictionary["future"] == []:
    data_in = dictionary["past"].float().to(device)
    data_out_time = None
    outputs = model_test(data_in,data_out_time)
  else:
    print(dictionary["future"][:,:,:1])
    data_out_time = dictionary["future"][:,:,:1].float().to(device)
    data_out_data = dictionary["future"][:,:,1:].float().to(device) 
    outputs = model_test(data_in,data_out_time)
    loss = criterion(outputs, data_out_data)
    print("loss: ", loss)
#    break